In [4]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import chi2_contingency, ttest_ind

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")



About Dataset
The EM-DAT - Country Profiles dataset provides aggregated figures for natural disasters in EM-DAT, which is the Emergency Events Database, maintained by the Centre for Research on the Epidemiology of Disasters (CRED). The dataset includes data on the number of disasters, the total number of people affected, the total number of deaths, and economic losses (original value and adjusted) for different disaster subtypes for each country and year. The dataset contains information on natural disasters from around the world.

Columns:
- Year: The year in which the disaster occurred.
- Country: The country where the disaster occurred.
- Disaster Subtype: The type of disaster (e.g., flood, earthquake, storm, drought).
- Number of Disasters: The total number of disasters that occurred in a given year for a specific country and disaster subtype.
- Total Affected: The total number of people affected by the disaster in a given year for a specific country and disaster subtype.
- Total Deaths: The total number of deaths resulting from the disaster in a given year for a specific country and disaster subtype.
- Total Economic Losses (Original): The total economic losses resulting from the disaster, in the original currency of the country, in a given year for a specific country and disaster subtype.
- Total Economic Losses (Adjusted): The total economic losses resulting from the disaster, adjusted for inflation and expressed in 2019 US dollars, in a given year for a specific country and disaster subtype.

Data volume:
The dataset contains multiple rows of data for each combination of year, country, and disaster subtype, spanning from 1900 to 2022. The dataset has a total of 47,682 rows and 8 columns.

Link al dataset:
- https://www.kaggle.com/datasets/mexwell/natural-disasters-emergency-events-database

In [13]:
# cargamos el dataframe correspondiente 
df = pd.read_csv("archivos/natural_disasters.csv", delimiter=';')

display(df.head())
display(df.tail())

,Year,Country,ISO,Disaster Group,Disaster Subroup,Disaster Type,Disaster Subtype,Total Events,Total Affected,Total Deaths,"Total Damage (USD, original)","Total Damage (USD, adjusted)",CPI
0,1900,Cabo Verde,CPV,Natural,Climatological,Drought,Drought,1,NaN,11000.0,NaN,NaN,"2,8490844088613"
1,1900,India,IND,Natural,Climatological,Drought,Drought,1,NaN,1250000.0,NaN,NaN,"2,8490844088613"
2,1900,Jamaica,JAM,Natural,Hydrological,Flood,NaN,1,NaN,300.0,NaN,NaN,"2,8490844088613"
3,1900,Japan,JPN,Natural,Geophysical,Volcanic activity,Ash fall,1,NaN,30.0,NaN,NaN,"2,8490844088613"
4,1900,Turkey,TUR,Natural,Geophysical,Earthquake,Ground movement,1,NaN,140.0,NaN,NaN,"2,8490844088613"


,Year,Country,ISO,Disaster Group,Disaster Subroup,Disaster Type,Disaster Subtype,Total Events,Total Affected,Total Deaths,"Total Damage (USD, original)","Total Damage (USD, adjusted)",CPI
10426,2023,Vanuatu,VUT,Natural,Meteorological,Storm,Tropical cyclone,2,502702.0,NaN,NaN,NaN,NaN
10427,2023,South Africa,ZAF,Natural,Hydrological,Flood,NaN,2,1500.0,20.0,NaN,NaN,NaN
10428,2023,Zambia,ZMB,Natural,Hydrological,Flood,NaN,1,22000.0,NaN,NaN,NaN,NaN
10429,2023,Zambia,ZMB,Natural,Hydrological,Flood,Flash flood,1,154608.0,NaN,NaN,NaN,NaN
10430,2023,Zimbabwe,ZWE,Natural,Meteorological,Storm,Tropical cyclone,1,NaN,NaN,NaN,NaN,NaN


In [14]:
def exploracion(df):
    df_info = pd.DataFrame()
    df_info["% nulos"] = round(df.isna().sum()/df.shape[0]*100, 2).astype(str)+"%"
    df_info["% no_nulos"] = round(df.notna().sum()/df.shape[0]*100, 2).astype(str)+"%"
    df_info["tipo_dato"] = df.dtypes
    df_info["num_valores_unicos"] = df.nunique()
    print(f"""El DataFrame tiene {df.shape[0]} filas y {df.shape[1]} columnas.
Tiene {df.duplicated().sum()} datos duplicados, lo que supone un porcentaje de {round(df.duplicated().sum()/df.shape[0], 2)}% de los datos.
Hay {len(list(df_info[(df_info["% nulos"] != "0.0%")].index))} columnas con datos nulos, y son:
{list(df_info[(df_info["% nulos"] != "0.0%")].index)}
y sin nulos hay {len(list(df_info[(df_info["% nulos"] == "0.0%")].index))} columnas y son:
{list(df_info[(df_info["% nulos"] == "0.0%")].index)}
A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:""")
    display(df_info.head())
    print("Principales estadísticos de las columnas categóricas:")
    display(df.describe(include="O").T)
    print("Principales estadísticos de las columnas numéricas:")
    display(df.describe(exclude="O").T)
    return df_info

exploracion(df)

El DataFrame tiene 10431 filas y 13 columnas.
Tiene 0 datos duplicados, lo que supone un porcentaje de 0.0% de los datos.
Hay 6 columnas con datos nulos, y son:
['Disaster Subtype', 'Total Affected', 'Total Deaths', 'Total Damage (USD, original)', 'Total Damage (USD, adjusted)', 'CPI']
y sin nulos hay 7 columnas y son:
['Year', 'Country', 'ISO', 'Disaster Group', 'Disaster Subroup', 'Disaster Type', 'Total Events']
A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Year,0.0%,100.0%,int64,124
Country,0.0%,100.0%,object,225
ISO,0.0%,100.0%,object,225
Disaster Group,0.0%,100.0%,object,1
Disaster Subroup,0.0%,100.0%,object,5


Principales estadísticos de las columnas categóricas:


,count,unique,top,freq
Country,10431,225,United States of America (the),405
ISO,10431,225,USA,405
Disaster Group,10431,1,Natural,10431
Disaster Subroup,10431,5,Hydrological,4489
Disaster Type,10431,13,Flood,3837
Disaster Subtype,8298,25,Riverine flood,1628
CPI,10380,114,"61,4631882611914",295


Principales estadísticos de las columnas numéricas:


,count,mean,std,min,25%,50%,75%,max
Year,10431.0,1.995610e+03,2.200119e+01,1900.0,1986.0,2001.0,2011.0,2.023000e+03
Total Events,10431.0,1.446649e+00,1.246589e+00,1.0,1.0,1.0,1.0,2.000000e+01
Total Affected,7586.0,1.125969e+06,9.760891e+06,1.0,1200.0,11414.0,119304.5,3.300000e+08
Total Deaths,7375.0,3.107711e+03,7.255589e+04,1.0,6.0,23.0,90.0,3.700000e+06
"Total Damage (USD, original)",3834.0,1.122262e+09,6.792339e+09,2000.0,10000000.0,68000000.0,400000000.0,2.100000e+11
"Total Damage (USD, adjusted)",3830.0,1.748704e+09,9.115319e+09,2469.0,20209265.5,146924694.0,784776702.0,2.732184e+11


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Year,0.0%,100.0%,int64,124
Country,0.0%,100.0%,object,225
ISO,0.0%,100.0%,object,225
Disaster Group,0.0%,100.0%,object,1
Disaster Subroup,0.0%,100.0%,object,5
Disaster Type,0.0%,100.0%,object,13
Disaster Subtype,20.45%,79.55%,object,25
Total Events,0.0%,100.0%,int64,19
Total Affected,27.27%,72.73%,float64,4043
Total Deaths,29.3%,70.7%,float64,815


# Valores unicos de las variables categoricas ✨

In [35]:
# creamos una lista con los nombres de las columnas categoricas 
columnas = df.select_dtypes(include='object').columns.tolist()
print(columnas)
# empezamos a iterar por cada una de las columnas para sacar sus valores únicos y sus frecuencias
for columna in columnas:
    print(f" \n----------- ESTAMOS ANALIZANDO LA COLUMNA: '{columna.upper()}' -----------\n")
    print(f"Sus valores únicos son: {df[columna].unique()}\n")
    print(f"Las frecuencias de los valores únicos de las categorías son: {df[columna].value_counts()} ")

['Country', 'ISO', 'Disaster Group', 'Disaster Subroup', 'Disaster Type', 'Disaster Subtype', 'CPI', 'continent']
 
----------- ESTAMOS ANALIZANDO LA COLUMNA: 'COUNTRY' -----------

Sus valores únicos son: ['Cabo Verde' 'India' 'Jamaica' 'Japan' 'Turkey'
 'United States of America (the)' 'China' 'Guatemala' 'Myanmar'
 'Martinique' 'Soviet Union' 'Saint Vincent and the Grenadines' 'Canada'
 'Comoros (the)' 'Iran (Islamic Republic of)' 'Israel' 'Niger (the)'
 'Bangladesh' 'Greece' 'Taiwan (Province of China)' 'Albania' 'Italy'
 'Philippines (the)' 'Belgium' 'Chile' 'Colombia' 'Hong Kong' 'Romania'
 'France' 'Haiti' 'Morocco' 'Pakistan' 'Portugal' 'Burkina Faso'
 'Costa Rica' 'Algeria' 'Gambia (the)' 'Guinea-Bissau' 'Mali' 'Mauritania'
 'Senegal' 'Chad' 'Kazakhstan' 'Mexico' 'Indonesia' 'Peru' 'Tokelau'
 'Puerto Rico' 'Anguilla' 'Argentina' 'Germany Fed Rep' 'Ecuador'
 'Bahamas (the)' 'Cuba' 'Egypt' 'Jordan' 'Bulgaria' 'Guadeloupe'
 'Saint Kitts and Nevis' 'Montserrat' 'Poland' 'New Zeala

# Creamos columna CONTINENTE

In [27]:
country_to_continent = {
    # Africa
    'Cabo Verde': 'Africa', 'Comoros (the)': 'Africa', 'Algeria': 'Africa', 'Gambia (the)': 'Africa',
    'Guinea-Bissau': 'Africa', 'Mali': 'Africa', 'Mauritania': 'Africa', 'Senegal': 'Africa',
    'Chad': 'Africa', 'Tunisia': 'Africa', 'Ethiopia': 'Africa', 'Somalia': 'Africa',
    'Kenya': 'Africa', 'Tanzania, United Republic of': 'Africa', 'Botswana': 'Africa',
    'Congo (the Democratic Republic of the)': 'Africa', 'Malawi': 'Africa', 'Nigeria': 'Africa',
    'South Africa': 'Africa', 'Sudan (the)': 'Africa', 'Zimbabwe': 'Africa', 'Angola': 'Africa',
    'Namibia': 'Africa', 'Eswatini': 'Africa', 'Liberia': 'Africa', 'Seychelles': 'Africa',
    'Libya': 'Africa', 'Mozambique': 'Africa', 'Madagascar': 'Africa', 'Togo': 'Africa',
    'Uganda': 'Africa', 'Benin': 'Africa', 'Cameroon': 'Africa', 'Burundi': 'Africa',
    'Rwanda': 'Africa', 'Djibouti': 'Africa', 'Zambia': 'Africa', 'Guinea': 'Africa',
    'Côte d’Ivoire': 'Africa', 'Gabon': 'Africa', 'Eritrea': 'Africa',
    'Sao Tome and Principe': 'Africa', 'Central African Republic': 'Africa',
    
    # Americas
    'United States of America (the)': 'Americas', 'Canada': 'Americas', 'Mexico': 'Americas',
    'Brazil': 'Americas', 'Argentina': 'Americas', 'Chile': 'Americas', 'Colombia': 'Americas',
    'Peru': 'Americas', 'Venezuela (Bolivarian Republic of)': 'Americas', 'Ecuador': 'Americas',
    'Costa Rica': 'Americas', 'Guatemala': 'Americas', 'Puerto Rico': 'Americas',
    'Cuba': 'Americas', 'Bahamas (the)': 'Americas', 'Haiti': 'Americas', 
    'Dominican Republic (the)': 'Americas', 'Belize': 'Americas', 'Honduras': 'Americas',
    'El Salvador': 'Americas', 'Panama': 'Americas', 'Jamaica': 'Americas', 
    'Barbados': 'Americas', 'Trinidad and Tobago': 'Americas', 
    'Grenada': 'Americas', 'Saint Vincent and the Grenadines': 'Americas', 
    'Saint Kitts and Nevis': 'Americas', 'Antigua and Barbuda': 'Americas', 
    'Paraguay': 'Americas', 'Bolivia (Plurinational State of)': 'Americas', 
    'Guyana': 'Americas', 'Suriname': 'Americas',
    
    # Asia
    'China': 'Asia', 'India': 'Asia', 'Japan': 'Asia', 'Pakistan': 'Asia', 'Bangladesh': 'Asia',
    'Korea (the Republic of)': 'Asia', 'Indonesia': 'Asia', 'Malaysia': 'Asia',
    'Philippines (the)': 'Asia', 'Vietnam': 'Asia', 'Thailand': 'Asia', 'Afghanistan': 'Asia',
    'Myanmar': 'Asia', 'Kazakhstan': 'Asia', 'Armenia': 'Asia', 'Lebanon': 'Asia',
    'Sri Lanka': 'Asia', "Lao People's Democratic Republic (the)": 'Asia', 
    'Cambodia': 'Asia', 'Nepal': 'Asia', 'Georgia': 'Asia', 'Mongolia': 'Asia', 
    'Bhutan': 'Asia', 'Azerbaijan': 'Asia', 'Uzbekistan': 'Asia', 'Tajikistan': 'Asia',
    'Kyrgyzstan': 'Asia', 'Turkmenistan': 'Asia', 'Maldives': 'Asia', 
    'Brunei Darussalam': 'Asia', 'Timor-Leste': 'Asia',
    
    # Europe
    'France': 'Europe', 'Germany': 'Europe', 'Italy': 'Europe', 'United Kingdom of Great Britain and Northern Ireland (the)': 'Europe',
    'Spain': 'Europe', 'Portugal': 'Europe', 'Belgium': 'Europe', 'Netherlands (the)': 'Europe',
    'Ireland': 'Europe', 'Luxembourg': 'Europe', 'Switzerland': 'Europe', 'Austria': 'Europe',
    'Denmark': 'Europe', 'Norway': 'Europe', 'Sweden': 'Europe', 'Finland': 'Europe',
    'Greece': 'Europe', 'Romania': 'Europe', 'Albania': 'Europe', 'Hungary': 'Europe',
    'Poland': 'Europe', 'Iceland': 'Europe', 'Czech Republic (the)': 'Europe',
    'Slovakia': 'Europe', 'Latvia': 'Europe', 'Lithuania': 'Europe', 'Slovenia': 'Europe',
    'Croatia': 'Europe', 'Bosnia and Herzegovina': 'Europe', 'Serbia': 'Europe',
    'Montenegro': 'Europe', 'Bulgaria': 'Europe', 'Estonia': 'Europe',
    
    # Oceania
    'Australia': 'Oceania', 'New Zealand': 'Oceania', 'Fiji': 'Oceania', 'Vanuatu': 'Oceania',
    'Papua New Guinea': 'Oceania', 'Tonga': 'Oceania', 'Samoa': 'Oceania', 'Kiribati': 'Oceania',
    'Solomon Islands': 'Oceania', 'Micronesia (Federated States of)': 'Oceania',
    'Marshall Islands (the)': 'Oceania', 'Tuvalu': 'Oceania', 'Palau': 'Oceania',
    'American Samoa': 'Oceania', 'Cook Islands (the)': 'Oceania', 'Niue': 'Oceania',
    'Tokelau': 'Oceania', 'Wallis and Futuna': 'Oceania',
    
    # Middle East
    'Iran (Islamic Republic of)': 'Middle East', 'Israel': 'Middle East', 'Saudi Arabia': 'Middle East',
    'Jordan': 'Middle East', 'Turkey': 'Middle East', 'Cyprus': 'Middle East', 'Iraq': 'Middle East',
    'Kuwait': 'Middle East', 'United Arab Emirates (the)': 'Middle East', 'Qatar': 'Middle East',
    'Palestine, State of': 'Middle East', 'Yemen': 'Middle East', 'Oman': 'Middle East',
    'Syrian Arab Republic': 'Middle East',
    
    # Unknown or regions
    'Soviet Union': 'Unknown', 'Yugoslavia': 'Unknown', 'German Fed Rep': 'Unknown',
    'Azores Islands': 'Unknown', 'French Polynesia': 'Unknown',
    'Réunion': 'Unknown', 'Canary Is': 'Unknown',
    'Saint Martin (French Part)': 'Unknown', 'Sint Maarten (Dutch part)': 'Unknown'
}


In [28]:

# Crear nueva columna 'continent' basada en 'Country'
df['continent'] = df['Country'].map(country_to_continent)

# Mostrar un ejemplo
display(df[['Country', 'continent']].head())


,Country,continent
0,Cabo Verde,Africa
1,India,Asia
2,Jamaica,Americas
3,Japan,Asia
4,Turkey,Middle East


In [36]:
df.head(10)

,Year,Country,ISO,Disaster Group,Disaster Subroup,Disaster Type,Disaster Subtype,Total Events,Total Affected,Total Deaths,"Total Damage (USD, original)","Total Damage (USD, adjusted)",CPI,continent
0,1900,Cabo Verde,CPV,Natural,Climatological,Drought,Drought,1,NaN,11000.0,NaN,NaN,"2,8490844088613",Africa
1,1900,India,IND,Natural,Climatological,Drought,Drought,1,NaN,1250000.0,NaN,NaN,"2,8490844088613",Asia
2,1900,Jamaica,JAM,Natural,Hydrological,Flood,NaN,1,NaN,300.0,NaN,NaN,"2,8490844088613",Americas
3,1900,Japan,JPN,Natural,Geophysical,Volcanic activity,Ash fall,1,NaN,30.0,NaN,NaN,"2,8490844088613",Asia
4,1900,Turkey,TUR,Natural,Geophysical,Earthquake,Ground movement,1,NaN,140.0,NaN,NaN,"2,8490844088613",Middle East
5,1900,United States of America (the),USA,Natural,Meteorological,Storm,Tropical cyclone,1,NaN,6000.0,30000000.0,1.052970e+09,"2,8490844088613",Americas
6,1901,Japan,JPN,Natural,Geophysical,Earthquake,Tsunami,1,24.0,18.0,NaN,NaN,"2,8490844088613",Asia
7,1902,China,CHN,Natural,Geophysical,Earthquake,Ground movement,1,NaN,2500.0,NaN,NaN,"2,9630477852157",Asia
8,1902,Guatemala,GTM,Natural,Geophysical,Earthquake,Ground movement,1,NaN,2000.0,25000000.0,8.437258e+08,"2,9630477852157",Americas
9,1902,Guatemala,GTM,Natural,Geophysical,Volcanic activity,Ash fall,2,NaN,7000.0,NaN,NaN,"2,9630477852157",Americas


In [29]:
exploracion(df)

El DataFrame tiene 10431 filas y 14 columnas.
Tiene 0 datos duplicados, lo que supone un porcentaje de 0.0% de los datos.
Hay 7 columnas con datos nulos, y son:
['Disaster Subtype', 'Total Affected', 'Total Deaths', 'Total Damage (USD, original)', 'Total Damage (USD, adjusted)', 'CPI', 'continent']
y sin nulos hay 7 columnas y son:
['Year', 'Country', 'ISO', 'Disaster Group', 'Disaster Subroup', 'Disaster Type', 'Total Events']
A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Year,0.0%,100.0%,int64,124
Country,0.0%,100.0%,object,225
ISO,0.0%,100.0%,object,225
Disaster Group,0.0%,100.0%,object,1
Disaster Subroup,0.0%,100.0%,object,5


Principales estadísticos de las columnas categóricas:


,count,unique,top,freq
Country,10431,225,United States of America (the),405
ISO,10431,225,USA,405
Disaster Group,10431,1,Natural,10431
Disaster Subroup,10431,5,Hydrological,4489
Disaster Type,10431,13,Flood,3837
Disaster Subtype,8298,25,Riverine flood,1628
CPI,10380,114,"61,4631882611914",295
continent,9308,7,Asia,2928


Principales estadísticos de las columnas numéricas:


,count,mean,std,min,25%,50%,75%,max
Year,10431.0,1.995610e+03,2.200119e+01,1900.0,1986.0,2001.0,2011.0,2.023000e+03
Total Events,10431.0,1.446649e+00,1.246589e+00,1.0,1.0,1.0,1.0,2.000000e+01
Total Affected,7586.0,1.125969e+06,9.760891e+06,1.0,1200.0,11414.0,119304.5,3.300000e+08
Total Deaths,7375.0,3.107711e+03,7.255589e+04,1.0,6.0,23.0,90.0,3.700000e+06
"Total Damage (USD, original)",3834.0,1.122262e+09,6.792339e+09,2000.0,10000000.0,68000000.0,400000000.0,2.100000e+11
"Total Damage (USD, adjusted)",3830.0,1.748704e+09,9.115319e+09,2469.0,20209265.5,146924694.0,784776702.0,2.732184e+11


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Year,0.0%,100.0%,int64,124
Country,0.0%,100.0%,object,225
ISO,0.0%,100.0%,object,225
Disaster Group,0.0%,100.0%,object,1
Disaster Subroup,0.0%,100.0%,object,5
Disaster Type,0.0%,100.0%,object,13
Disaster Subtype,20.45%,79.55%,object,25
Total Events,0.0%,100.0%,int64,19
Total Affected,27.27%,72.73%,float64,4043
Total Deaths,29.3%,70.7%,float64,815


# Vemos los nulos
- Nota: ya he actualizado la lista para que haya menos nulos ✨

In [31]:
pd.set_option('display.max_rows', None)

In [32]:
# Filtrar filas donde el continente es nulo
rows_with_null_continent = df[df['continent'].isnull()]

# Mostrar las filas con valores nulos en 'continent'
display(rows_with_null_continent[['Country', 'continent']])

,Country,continent
12,Martinique,NaN
20,Niger (the),NaN
26,Taiwan (Province of China),NaN
39,Hong Kong,NaN
41,Niger (the),NaN
42,Taiwan (Province of China),NaN
57,Morocco,NaN
60,Taiwan (Province of China),NaN
63,Burkina Faso,NaN
73,Niger (the),NaN


In [33]:
# Valores unicos de paises nulos
print(f"Sus valores únicos son: {rows_with_null_continent['Country'].unique()}\n")


Sus valores únicos son: ['Martinique' 'Niger (the)' 'Taiwan (Province of China)' 'Hong Kong'
 'Morocco' 'Burkina Faso' 'Anguilla' 'Germany Fed Rep' 'Egypt'
 'Guadeloupe' 'Montserrat' 'Dominica' 'Nicaragua' 'Ghana' 'New Caledonia'
 'Yemen Arab Rep' 'Bermuda' 'Viet Nam' 'Netherlands Antilles'
 'Saint Lucia' 'Mauritius' 'Guam' 'Congo (the)' 'Uruguay' 'Lesotho'
 'Yemen P Dem Rep' 'Czechoslovakia' 'Sierra Leone' 'Germany Dem Rep'
 'Turks and Caicos Islands (the)'
 "Korea (the Democratic People's Republic of)" 'Virgin Island (U.S.)'
 'Russian Federation (the)' 'Serbia Montenegro' 'Ukraine' 'Belarus'
 'Macao' 'Macedonia (the former Yugoslav Republic of)'
 'Moldova (the Republic of)' 'French Guiana' 'Virgin Island (British)'
 'Cayman Islands (the)' 'Saint Helena, Ascension and Tristan da Cunha'
 'Northern Mariana Islands (the)' 'South Sudan' 'Saint Barthélemy'
 'Isle of Man']

